In [15]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.setrecursionlimit(1000000)
!pip3 uninstall torch torchvision torchaudio 

Found existing installation: torch 1.11.0+cu113
Uninstalling torch-1.11.0+cu113:
  Would remove:
    /home/danish/.local/bin/convert-caffe2-to-onnx
    /home/danish/.local/bin/convert-onnx-to-caffe2
    /home/danish/.local/bin/torchrun
    /home/danish/.local/lib/python3.10/site-packages/caffe2/*
    /home/danish/.local/lib/python3.10/site-packages/torch-1.11.0+cu113.dist-info/*
    /home/danish/.local/lib/python3.10/site-packages/torch/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [2]:
id =[]
sequence = []
name_to_seq_dict = {}
from Bio import SeqIO
for seq_record in SeqIO.parse("9606.protein.sequences.v11.5.fa", "fasta"):
    id.append(seq_record.id)
    x = repr(seq_record.seq)[5:-2]
    l = ""
    for i in x:
        l = l + i + " "
    l = l.rstrip()
    
    sequence.append(l)
name_to_seq_dict = dict(zip(id, sequence))
name_to_seq_dict['9606.ENSP00000002829']
len(name_to_seq_dict) 

19566

In [4]:
data = pd.read_csv("ppi_dataset_from_positive_pair_string.csv")
data.head()
data['protein1_sequence'] = data.apply (lambda row: name_to_seq_dict[row['protein1']], axis=1)
data['protein2_sequence'] = data.apply (lambda row: name_to_seq_dict[row['protein2']], axis=1)

In [11]:
import torch
print(torch.cuda.is_available())

# print('Device name:', torch.cuda.get_device_name(0)


False


In [8]:
data_2 = pd.DataFrame()
data = data[['protein1_sequence','protein2_sequence','label']]
data['concat_protein'] = data['protein1_sequence'] +" " + data['protein2_sequence']
data_2['concat_protein'] = + data['protein2_sequence']+" " + data['protein1_sequence'] 
data_2['label'] = data['label']
data = data[['concat_protein','label']]
data_2 = data_2[['concat_protein','label']]

In [6]:
final_data = pd.concat([data,data_2]).reset_index(drop = True)

In [7]:
from sklearn.model_selection import train_test_split
X  = final_data.drop(['label'],axis=1).values
y = final_data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [8]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('Rostlab/prot_bert_bfd', do_lower_case=False )

In [9]:
def preprocessing_for_bert(protein_sequences,MAX_LEN = 122):
    input_ids = []
    attention_masks = []
    for protein_sequence in tqdm(protein_sequences):
        id = tokenizer.batch_encode_plus(protein_sequence ,max_length = MAX_LEN, add_special_tokens=True, pad_to_max_length=True)
        input_id = id.get('input_ids')
        attention_mask = id.get('attention_mask')
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
    
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)
    
    return input_ids, attention_masks

In [10]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
y_train = torch.tensor(y_train)
y_test = torch.tensor(y_test)
y_test = y_test[0:10000]

In [11]:
test_inputs,test_masks = preprocessing_for_bert(X_test[0:10000])

  0%|          | 0/10000 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/danish/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████| 10000/10000 [00:05<00:00, 1705.01

In [12]:
batch_size = 2
# train_data = TensorDataset(train_inputs, train_masks,y_train)
# train_sampler = RandomSampler(train_data)
# train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = batch_size,drop_last=True)

test_data = TensorDataset(test_inputs, test_masks,y_test)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size = batch_size,drop_last=True)

In [13]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [14]:
device

'cpu'

In [15]:
class BertClassifier(nn.Module):
    
    def __init__(self, embed_dim = 1024):
        super(BertClassifier,self).__init__()
        self.bert = BertModel.from_pretrained("Rostlab/prot_bert_bfd")
        
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(embed_dim,512)
        self.fc2 = nn.Linear(512,256)
        
        self.fc3 = nn.Linear(512,64)
        self.fc4 = nn.Linear(64,32)
        
        self.fc5 = nn.Linear(32,16)
        self.fc6 = nn.Linear(16,8)
        self.fc7 = nn.Linear(8,1)

    
    def forward(self, input_ids,attention_mask):
        
        embedding = self.bert(input_ids=input_ids,attention_mask=attention_mask)
        output = torch.tensor(embedding[0][:, 0, :])
        # print(output)
        output_1 = self.relu(self.fc1(output))
        output_2 = self.relu(self.fc3(output_1))
        output_3 = self.relu(self.fc4(output_2))
        output_4 = self.relu(self.fc5(output_3))
        output_5 = self.relu(self.fc6(output_4))
        output = self.fc7(output_5)

        return output

In [16]:
import torch.optim as optim

def initilize_model(learning_rate=0.001):
    """Instantiate a CNN model and an optimizer."""

    model = BertClassifier()
    
    device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
    # Send model to `device` (GPU/CPU)
    model.to(device)

    
    return model

In [ ]:
model = initilize_model(learning_rate=0.01)
checkpoint = torch.load('best_mode_trained_after_37_epochs.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 15.90 GiB total capacity; 845.42 MiB already allocated; 18.56 MiB free; 860.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
model.eval()
    
val_accuracy = []
val_loss = []
for batch in tqdm(test_dataloader):
        # Load batch to GPU
    b_input_ids, b_attn_masks, b_labels = tuple(t.to(device) for t in batch)
    b_input_ids = b_input_ids.reshape((1,48,122)).squeeze(0)
    b_attn_masks = b_attn_masks.reshape((1,48,122)).squeeze(0)
    b_labels = b_labels.reshape((1,48,1)).squeeze(0)
    with torch.no_grad():
        logits = model( b_input_ids, b_attn_masks)
        
        
        
    
    loss = loss_fn(logits, b_labels.float())

    val_loss.append(loss.item())
# # model = BertModel.from_pretrained("Rostlab/prot_bert_bfd")
# for i in tqdm(train_dataloader):
#     print(i)
    preds = torch.round(torch.sigmoid(logits))
        # print(preds,b_labels)
    accuracy = (preds.float() == b_labels.float()).cpu().numpy().mean() * 100

    val_accuracy.append(accuracy)
    
val_loss = np.mean(val_loss)
val_accuracy = np.mean(val_accuracy)
print(val_accuracy)